### Instructions
A - above
B - below
X - delete 
Ctrl Enter - run
Enter - Enter into cell
M - make cell into markdown cell

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot

# TASK 1 - MACHINE LEARNING

# TASK 2 - OPTIMISATION

In [3]:
def generate_timetable():
    """
    Generates a timetable.
    Read the file to see how many classes + labs there are.
    Store them in a 20x3 list alongside empty sessions to fill the list
    """
    #Create "empty" timetable
    timetable = [["empty" for x in range(3)] for y in range(20)] 
    
    #Read file into allSplits list
    allSplits = []
    f = open("modules.txt", 'r')
    lineCount = 0
    for line in f:
        allSplits.append(line.split("|"))
        lineCount+=1
    f.close()
    
    #Add module into timetable
    for i in range(0,lineCount):
        timetable[i][0] = allSplits[i][0]
    
    
    #Add seminars into timetable.
    #Iterate through each module, for each number of seminars use
    #counters to iterate to next empty position and place seminar.
    #Should work on different set of modules if format is kept identical.
    emptyChecker = 0
    emptyChecker1 = 0
    for i in range(0, lineCount):
        for j in range(0, int(allSplits[i][2])):
            toggle = False
            while toggle == False:
                if timetable[emptyChecker][emptyChecker1] == "empty":
                    timetable[emptyChecker][emptyChecker1] = allSplits[i][0]
                    toggle = True
                emptyChecker1+=1
                if emptyChecker1 == 3:
                    emptyChecker+=1
                    emptyChecker1=0
    
    #randomises through randomise function below
    #randomSwitch = False
    #while randomSwitch == False:
    randomise_timetable(timetable)
        #randomSwitch = room_validity(timetable)
    return timetable

In [11]:
def randomise_timetable(timetable):
    #Create randomised timetable
    #Add lectures to lec. list and labs to lab list
    #Shuffle lists
    #Put back into timetable
    
    lecturesJoined = []
    labsJoined = []
    for i in range(len(timetable)):
        for j in range(0,len(timetable[i]), len(timetable[i])):
            lecturesJoined.append(timetable[i][j])
            labsJoined.append(timetable[i][j+1])
            labsJoined.append(timetable[i][j+2])
    np.random.shuffle(lecturesJoined)
    np.random.shuffle(labsJoined)
    counterLecs = 0
    counterLabs = 0
    for i in range(0,len(timetable)):
        for j in range(0,len(timetable[i]), len(timetable[i])):
            timetable[i][j] = lecturesJoined[counterLecs]
            timetable[i][j+1] = labsJoined[counterLabs]
            timetable[i][j+2] = labsJoined[counterLabs+1]
            counterLecs+=1
            counterLabs+=2

    return timetable

In [19]:
def fitness_function(timetable):
    #Calculates fitness of given timetable
    
    allSplits = []
    f = open("modules.txt", 'r')
    lineCount = 0
    for line in f:
        allSplits.append(line.split("|"))
        lineCount+=1
    f.close()
    
    modSplits = []
    modConstraintSplits = []
    for line in allSplits:
        modSplits.append(line[0])
        modConstraintSplits.append(line[3])
    """
    CONSTRAINTS:
    1) ONLY 1 LECTURE AND TWO LABS MAY RUN CONCURRENTLY
    2) LABS FOR A MODULE MAY ONLY OCCUR AFTER ITS LECTURE! PRECEDENCE!
    3) SESSIONS CAN'T INTERFERE! CONCURRENCE!
    """
    #Precedence constraints
    precConstraints = 1
    #module loop
    for i in range(0, len(timetable)):
        module = timetable[i][0]
        if module == "empty":
            continue
        #all of the timetable lab loop
        for j in range(0, len(timetable)):
            if j > i:
                break
            for l in range(1, len(timetable[j])):
                if module == timetable[j][l]:
                    precConstraints+=1
            
    
    #Concurrency constraints
    #Functional and clean, aside from try catch section which only caters to list length 3.
    concConstraints = 1
    #Cycle through each session
    for i in range(0,len(timetable)):
        #Cycle through each room
        for j in range(0,len(timetable[i])):
            if timetable[i][j] == "empty":
                continue
            constraintModules = []
            pos1 = modSplits.index(timetable[i][j])
            clashes1 = modConstraintSplits[pos1]
            #2nd cycle through each room, linear comparison of modules and clashes
            for k in range(j,len(timetable[i])):
                if k == j or timetable[i][k] == "empty":
                    continue
                else:
                    pos2 = modSplits.index(timetable[i][k])
                    clashes2 = modConstraintSplits[pos2]
                    if timetable[i][j] == timetable[i][k] or timetable[i][j] in clashes2 or timetable[i][k] in clashes1:
                        concConstraints+=1

    return precConstraints * concConstraints

In [17]:
def mutation_swap(timetable):
    #This function swaps two modules at random.
    #Must be modified for another timetable as it's specific to
    #this problem and its room (1 theatre 2 lab) constraints. 
    moduleIndex = []
    tim = len(timetable)-1
    tim2 = len(timetable[0])-1
    moduleIndex.append(np.random.randint(0,tim))
    moduleIndex.append(np.random.randint(0,tim2)) 
    whileToggle = False
    x = np.random.randint(0,tim)
    if moduleIndex[1] == 0:
        y = 0
    else:
        y = np.random.randint(1, tim2) 
    while whileToggle == False:
        if x == moduleIndex[0] and y == moduleIndex[1]:
            x = np.random.randint(0,tim)
            y = np.random.randint(0,tim2)
        else:
            whileToggle = True
    moduleIndex.append(x)
    moduleIndex.append(y)
    mod1 = timetable[moduleIndex[0]][moduleIndex[1]]
    mod2 = timetable[moduleIndex[2]][moduleIndex[3]]
    timetable[moduleIndex[0]][moduleIndex[1]] = mod2
    timetable[moduleIndex[2]][moduleIndex[3]] = mod1
    return timetable
    

In [7]:
def mutation_ruin_and_run(timetable):
    #randomises given timetable
    return randomise_timetable(timetable)

In [8]:
#Objective Function for current coursework problem.
def comparison_lower(a, b):
    return b <= a

In [9]:
def hillclimber(itInt, timetable, problem, mutation, comparison):
    #itInt = no. of iterations
    #problem = fitness calculator
    #mutation = function by which to change decision variable
    #decision variable is the timetable, generated within the function!
    #comparison = objective function for current optimisation goal
    
    parent = timetable
    #for i in range(0,len(parent)):
        #print(parent[i])
    parentFitness = problem(parent)
    
    #Loop for itint no. of times
    fitnessList = []
    for iter in range(itInt):
        #if iter % 100 == 0:
            #print(f"Progress: {iter} / {itInt}")
        #Mutate
        child = mutation(parent)
        
        #Evaluate
        childFitness = problem(child)
        
        #Print parent soln
        if comparison(parentFitness, childFitness):
            #if parentFitness > childFitness:
                #print(f"New best! Fitness = {childFitness}")
            parent = child
            parentFitness = childFitness
        fitnessList.append(parentFitness)
    
    return parent, fitnessList

In [21]:
#Optimisation Algorithm Implementation
t = generate_timetable()
fi = open("results500v4.txt", "w")

for i in range(30):
    print(f"Progress: {i}/30")
    #Swap Mutation 
    swapFits = []
    x, y = hillclimber(500, t, fitness_function, mutation_swap, comparison_lower)
    print(f"Best Solution:")
    for i in range(0, len(x)):
        print(x[i])
    print(f"fitness: {y[len(y)-1]}")
    fi.writelines(str(x))
    fi.write(f"\n Fitness of run {i}: {y[len(y)-1]} \n")
    swapFits.append(y)

    #Ruin-and-Run Mutation
    rnrFits = []
    a, b = hillclimber(500, t, fitness_function, mutation_ruin_and_run, comparison_lower)
    print(f"Best Solution:")
    for i in range(0, len(a)):
        print(a[i])
    print(f"fitness: {b[len(b)-1]}")
    fi.writelines(str(a))
    fi.write(f"\n Fitness of run {i}: {b[len(b)-1]} \n")
    rnrFits.append(b)
    
fi.write("\n All Swap Fitnesses: ")
fi.write(str(swapFits))
fi.write("\n All RnR Fitnesses: ")
fi.write(str(rnrFits))
print("\nDone.")
fi.close()

Progress: 0/30
Best Solution:
['MOD012', 'MOD013', 'MOD010']
['empty', 'MOD001', 'MOD009']
['MOD003', 'MOD007', 'MOD014']
['MOD004', 'MOD005', 'empty']
['empty', 'empty', 'MOD011']
['MOD011', 'MOD015', 'empty']
['MOD017', 'MOD015', 'MOD012']
['MOD005', 'MOD013', 'empty']
['MOD002', 'MOD006', 'MOD008']
['MOD006', 'MOD017', 'MOD001']
['MOD002', 'MOD003', 'MOD006']
['empty', 'MOD016', 'MOD008']
['empty', 'MOD011', 'MOD015']
['MOD014', 'empty', 'MOD009']
['MOD010', 'MOD012', 'MOD016']
['empty', 'MOD001', 'MOD002']
['MOD014', 'MOD003', 'empty']
['MOD007', 'MOD010', 'empty']
['MOD009', 'MOD004', 'MOD009']
['MOD008', 'MOD005', 'MOD006']
fitness: 300
Best Solution:
['MOD002', 'MOD012', 'MOD009']
['MOD011', 'MOD006', 'MOD016']
['MOD002', 'MOD011', 'MOD010']
['MOD012', 'empty', 'MOD017']
['MOD010', 'MOD008', 'empty']
['empty', 'empty', 'empty']
['MOD008', 'empty', 'MOD003']
['MOD003', 'MOD005', 'MOD003']
['MOD009', 'MOD001', 'MOD008']
['MOD007', 'MOD001', 'MOD013']
['empty', 'MOD010', 'MOD015']


Best Solution:
['MOD006', 'empty', 'empty']
['MOD001', 'MOD002', 'MOD012']
['MOD002', 'MOD001', 'empty']
['MOD011', 'MOD009', 'empty']
['MOD005', 'MOD009', 'MOD006']
['MOD009', 'MOD011', 'empty']
['MOD014', 'MOD010', 'MOD017']
['MOD014', 'MOD003', 'MOD012']
['MOD006', 'empty', 'MOD015']
['empty', 'MOD003', 'MOD002']
['MOD016', 'MOD004', 'MOD006']
['MOD007', 'empty', 'MOD005']
['MOD014', 'MOD015', 'empty']
['MOD015', 'MOD017', 'MOD005']
['MOD008', 'empty', 'MOD012']
['empty', 'MOD010', 'MOD010']
['MOD013', 'MOD008', 'MOD016']
['MOD007', 'MOD013', 'MOD004']
['MOD011', 'MOD001', 'MOD008']
['MOD003', 'empty', 'MOD009']
fitness: 144
Progress: 7/30
Best Solution:
['MOD008', 'empty', 'empty']
['MOD016', 'MOD007', 'MOD012']
['MOD011', 'empty', 'empty']
['empty', 'MOD001', 'empty']
['MOD010', 'MOD003', 'MOD006']
['MOD010', 'MOD013', 'empty']
['MOD007', 'MOD009', 'MOD017']
['MOD009', 'empty', 'MOD012']
['MOD005', 'MOD011', 'MOD015']
['MOD014', 'MOD008', 'MOD002']
['MOD015', 'MOD009', 'MOD006']
[

Best Solution:
['MOD002', 'MOD002', 'MOD013']
['MOD004', 'empty', 'MOD007']
['MOD017', 'MOD006', 'MOD009']
['MOD003', 'MOD012', 'MOD005']
['MOD015', 'MOD016', 'empty']
['MOD010', 'empty', 'MOD005']
['MOD004', 'empty', 'MOD013']
['MOD005', 'MOD011', 'MOD011']
['empty', 'MOD015', 'MOD008']
['MOD009', 'empty', 'MOD008']
['MOD001', 'MOD016', 'MOD010']
['MOD014', 'MOD006', 'MOD015']
['MOD017', 'empty', 'MOD014']
['empty', 'MOD012', 'MOD006']
['MOD002', 'MOD012', 'MOD011']
['empty', 'MOD009', 'MOD001']
['MOD009', 'empty', 'MOD007']
['empty', 'MOD008', 'empty']
['MOD003', 'MOD001', 'MOD014']
['MOD006', 'MOD010', 'MOD003']
fitness: 216
Best Solution:
['MOD006', 'MOD010', 'MOD014']
['empty', 'MOD011', 'MOD013']
['MOD002', 'empty', 'MOD011']
['empty', 'MOD014', 'empty']
['MOD014', 'MOD011', 'MOD001']
['MOD002', 'MOD006', 'MOD010']
['MOD003', 'MOD006', 'MOD008']
['MOD005', 'MOD005', 'MOD007']
['MOD001', 'MOD016', 'MOD012']
['MOD017', 'MOD009', 'empty']
['MOD004', 'MOD015', 'MOD002']
['MOD004', 'e

['MOD014', 'MOD005', 'MOD004']
['MOD007', 'empty', 'MOD003']
['MOD001', 'MOD011', 'MOD015']
['MOD012', 'empty', 'MOD014']
['MOD011', 'empty', 'MOD002']
['MOD008', 'MOD011', 'MOD015']
['MOD004', 'MOD001', 'MOD009']
['MOD009', 'empty', 'MOD008']
['MOD009', 'MOD006', 'MOD010']
fitness: 210
Progress: 20/30
Best Solution:
['MOD012', 'empty', 'MOD013']
['empty', 'empty', 'MOD002']
['MOD002', 'empty', 'MOD006']
['MOD005', 'MOD006', 'MOD007']
['MOD010', 'empty', 'MOD014']
['MOD008', 'MOD009', 'MOD017']
['MOD011', 'MOD014', 'MOD003']
['MOD008', 'MOD017', 'MOD009']
['MOD016', 'empty', 'MOD005']
['MOD010', 'empty', 'MOD015']
['empty', 'MOD003', 'MOD006']
['MOD012', 'MOD011', 'MOD004']
['empty', 'MOD012', 'MOD003']
['MOD007', 'MOD011', 'MOD015']
['MOD005', 'empty', 'MOD014']
['MOD016', 'MOD004', 'MOD002']
['empty', 'empty', 'MOD015']
['MOD013', 'MOD001', 'MOD009']
['MOD001', 'MOD001', 'MOD008']
['MOD009', 'MOD006', 'MOD010']
fitness: 182
Best Solution:
['MOD007', 'MOD017', 'MOD011']
['MOD010', 'MO

Best Solution:
['MOD009', 'empty', 'MOD010']
['MOD001', 'empty', 'MOD003']
['MOD011', 'MOD003', 'MOD008']
['MOD014', 'MOD009', 'MOD006']
['MOD005', 'MOD013', 'empty']
['MOD013', 'MOD015', 'MOD006']
['MOD006', 'MOD009', 'MOD002']
['MOD007', 'MOD009', 'empty']
['MOD014', 'MOD014', 'MOD012']
['empty', 'MOD004', 'MOD005']
['empty', 'MOD015', 'MOD006']
['MOD001', 'MOD004', 'MOD001']
['MOD011', 'empty', 'MOD002']
['MOD017', 'MOD007', 'empty']
['MOD010', 'empty', 'MOD008']
['MOD003', 'MOD015', 'MOD016']
['MOD002', 'MOD016', 'empty']
['MOD010', 'MOD011', 'MOD017']
['empty', 'MOD005', 'MOD008']
['MOD012', 'MOD012', 'empty']
fitness: 168
Progress: 27/30
Best Solution:
['MOD013', 'MOD003', 'MOD010']
['MOD002', 'MOD014', 'MOD003']
['MOD007', 'MOD009', 'MOD008']
['MOD006', 'MOD011', 'MOD006']
['MOD013', 'MOD009', 'empty']
['MOD017', 'empty', 'MOD006']
['MOD001', 'empty', 'MOD002']
['MOD005', 'MOD003', 'empty']
['MOD009', 'empty', 'MOD012']
['MOD004', 'MOD014', 'MOD005']
['MOD015', 'MOD009', 'MOD006